# Classificador Random Forest - Implantação

Este componente realiza predições usando um modelo Random Forest para classificação usando [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html). <br>
Scikit-learn é uma biblioteca open source de machine learning que suporta apredizado supervisionado e não supervisionado. Também provê várias ferramentas para montagem de modelo, pré-processamento de dados, seleção e avaliação de modelos, e muitos outros utilitários.

Este notebook apresenta:
- como carregar modelos e outros resultados do treinamento.
- como utilizar um modelo para fornecer predições em tempo real.

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [ ]:
%%writefile Model.py
import joblib
import numpy as np
import pandas as pd


class Model(object):
    def __init__(self):
        # Carrega artefatos: estimador, etc
        artifacts = joblib.load("/tmp/data/random-forest-classifier.joblib")
        self.pipeline = artifacts["pipeline"]
        self.features_names_training = artifacts["columns"]
        self.columns_to_filter = artifacts["columns_to_filter"]
        self.label_encoder = artifacts["label_encoder"]
        self.method = artifacts["method"]
        self.new_columns = artifacts["new_columns"]
        self.features_after_pipeline = artifacts["features_after_pipeline"]

    def class_names(self):
        column_names = np.concatenate((self.features_after_pipeline, self.new_columns))
        return column_names.tolist()

    def predict(self, X, feature_names, meta=None):
        if feature_names:
            # Antes de utilizar o conjunto de dados X no modelo, reordena suas features de acordo com a ordem utilizada no treinamento
            df = pd.DataFrame(X, columns=feature_names)
            X = df[self.columns_to_filter].to_numpy()

        # adiciona a predição ao conjunto de dados
        y_prob = self.pipeline.predict_proba(X)
        y_pred = self.pipeline.predict(X)
        y_pred = self.label_encoder.inverse_transform(y_pred)
        for i in range(y_prob.shape[1]):
            df[self.new_columns[i]] = y_prob[:, i]
        df[self.new_columns[i + 1]] = y_pred

        return df.to_numpy()

## Teste do serviço REST

Crie um arquivo `contract.json` com os seguintes atributos:

- `features` : A lista de features em uma requisição.
- `targets` : A lista de valores retornados pelo método `predict`.

Cada `feature` pode conter as seguintes informações:

- `name` : nome da feature
- `ftype` : tipo da feature : **continuous** ou **categorical**
- `dtype` : tipo de dado : **FLOAT** ou **INT** : *obrigatório para ftype continuous*
- `range` : intervalo de valores numéricos : *obrigatório para ftype continuous*
- `values` : lista de valores categóricos : *obrigatório para ftype categorical*

Em seguida, utilize a função `test_deployment` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para simular predição em tempo-real.<br>

In [ ]:
%%writefile contract.json
{
    "features": [
        {
            "name": "SepalLengthCm",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [4.3, 7.9]
        },
        {
            "name": "SepalWidthCm",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [2.0, 4.4]
        },
        {
            "name": "PetalLengthCm",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [1.0, 6.9]
        },
        {
            "name": "PetalWidthCm",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.1, 2.5]
        }
    ],
    "targets": [
        {
            "name": "Iris-setosa",
            "ftype": "continuous",
            "dtype": "FLOAT",
            "range": [0.0, 1.0]
        },
                {
            "name": "Iris-versicolor",
            "ftype": "continuous",
            "dtype": "FLOAT",
            "range": [0.0, 1.0]
        },
                {
            "name": "Iris-virginica",
            "ftype": "continuous",
            "dtype": "FLOAT",
            "range": [0.0, 1.0]
        }
    ]
}

In [ ]:
from platiagro.deployment import test_deployment

test_deployment("contract.json")